In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv("data/client_knp.csv")
df.columns = ["client", "knp"]
df.knp = df.knp.str.replace(",", " ")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
df[df.client == "910130350727"]

,client,knp,w2v
795822,910130350727,321 321 321 321 321 321 321 321 321 311 321 32...,"[-3.1722819109914604, -0.21523173649385496, -2..."
1165142,910130350727,311 321 321 321,"[-3.349206695057366, 0.9078981178338144, -2.02..."


In [3]:
from gensim.models.word2vec import Word2Vec

word2vec = Word2Vec.load("data/w2v.model")
vocab = word2vec.wv.vocab.keys()
word_vectors = {key:word2vec.wv[key] for key in vocab}
vectors = [word2vec.wv[key] for key in vocab]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
_ = tfidf.fit_transform(df.knp)
idfs = {key:tfidf.idf_[value] for key, value in tfidf.vocabulary_.items()}

In [5]:
def get_vector(knps):
    knps = list(filter(lambda knp: knp in word_vectors.keys() and knp in idfs.keys(), knps.split()))
    
    if not knps:
        return np.nan
    
    tf = 1.0 / len(knps)
    weights = np.array([idfs[knp] * tf for knp in knps])
    knp_vecs = np.array([word_vectors[knp] for knp in knps])
    
    return np.multiply(knp_vecs, weights.reshape(-1, 1)).sum(axis=0)

df["w2v"] = df.knp.apply(get_vector)

In [6]:
df = df.loc[~df.w2v.isnull()]

In [23]:
df["w2v_str"] = df.w2v.apply(lambda x: ",".join(map(lambda y: "{0:.4f}".format(y), x)))

In [25]:
df[["client", "w2v_str"]].to_csv("data/client_w2v.csv", index=False)

In [ ]:
%%time 

from scipy.spatial.distance import cosine

def get_distance(w2v):
    subset = df.sample(1000)
    vects = np.vstack(subset.w2v.values)
    
    norma = np.sqrt(np.sum(vects ** 2, axis=1))
    normb = np.sqrt(np.sum(w2v ** 2))

    return (vects.dot(w2v) / (norma * normb)).mean()

distances = df.w2v.apply(get_distance)

In [16]:
clients_filters = pd.DataFrame({"client":df.client, "filter":distances})
clients_filters.to_csv("data/clients_filters.csv", index=False)

In [191]:
d = pd.read_csv("../data/off_members.csv")
с = pd.read_csv("../data/off_ops.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,5,11,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,8,9,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
distances

0          0.576029
1          0.532668
2          0.534769
3          0.531189
4          0.559371
5          0.541963
6          0.504647
7          0.534980
8          0.534061
9          0.492442
10         0.540061
11         0.546682
12         0.485075
13         0.451474
14         0.403788
15         0.443003
16         0.475790
17         0.176401
18        -0.286691
19         0.410627
20         0.599452
21         0.548893
22         0.533356
23         0.525004
24         0.365716
25         0.168120
26         0.485369
27        -0.272337
28         0.542681
29         0.420038
             ...   
1202842   -0.201900
1202843   -0.113080
1202844   -0.121621
1202845   -0.133847
1202846    0.006194
1202847   -0.124914
1202848   -0.100209
1202849   -0.130857
1202850   -0.165383
1202851    0.026093
1202852   -0.112494
1202853    0.025187
1202854   -0.114993
1202855   -0.098476
1202856   -0.218087
1202857   -0.120014
1202858   -0.206774
1202859   -0.113245
1202860    0.084375


In [202]:
a = d[d.P_CLIENTID == 5463014.0].merge(с, left_on="P_OPERATIONID", right_on="ID")

In [203]:
a.P_OPERATIONREASON

0    RETAIL PAYMENT; ЗАЧИСЛЕНИЕ КРЕДИТНЫХ СРЕДСТВ; ...
Name: P_OPERATIONREASON, dtype: object

AttributeError: 'DataFrame' object has no attribute 'client'

In [122]:
3.6 / 100 * df.shape[0] / 3600

12.028720000000002

In [46]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
mem_vars = sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
print(sum(map(lambda x: x[1], mem_vars)) / 1000 / 1000, "mb")

582.228786 mb


582.236354